In [1]:
import pandas as pd
import datetime as dt
import pandasql as ps
import os
import zipfile as zp
from datetime import datetime
#import matplotlib.pyplot as plt

DIR=r'C:\Users\anuj\Downloads'
ZIp='archive (4).zip'

os.chdir(DIR)
zf = zp.ZipFile(f'{ZIp}')

dfs={}
for name in zf.namelist():
    df_name= name.replace('.csv','')
    dfs[df_name]=pd.read_csv(zf.open(name,mode='r'),encoding='latin')

df1=dfs['ma_lga_12345']
df2=dfs['raw_sales']
df1.columns=df1.columns.str.replace(' ','_')
df2.columns=df2.columns.str.replace(' ','_')

#df1[df1.duplicated()]## Check duplicates
#df2[df2.duplicated()]## Check duplicates
#df2.info()
#df2.isnull().values.any() ########Check NUlls
#df1.isnull().isnull().sum() ########Check NUlls by each column
df2['datesold']= df2['datesold'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
df1['saledate']= df1['saledate'].apply(lambda x: datetime.strptime(x,'%d/%m/%Y'))
df2['postcode']= '0'+ df2['postcode'].apply(lambda x: str(x))
df2['bedrooms']= df2['bedrooms'].apply(lambda x: str(x))
df1['MA']= df1['MA'].apply(lambda x: str(x))
df1['bedrooms']= df1['bedrooms'].apply(lambda x: str(x))
merged=ps.sqldf('select * from df2 a left join df1 b on a.propertyType=b.type and a.bedrooms=b.bedrooms and a.datesold=b.saledate')
del df1,df2

In [5]:
merged['postcode'].unique()

array(['02607', '02906', '02905', '02606', '02902', '02612', '02904',
       '02615', '02914', '02602', '02600', '02605', '02603', '02611',
       '02903', '02617', '02913', '02604', '02614', '02912', '02601',
       '02900', '02620', '02618', '02616', '02911', '02609'], dtype=object)